# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tangalle,6.0240,80.7911,24.76,90,100,1.86,LK,1698002865
1,1,gadzhiyevo,69.2551,33.3362,0.13,71,97,7.50,RU,1698002865
2,2,albany,42.6001,-73.9662,10.23,72,100,1.34,US,1698002866
3,3,grytviken,-54.2811,-36.5092,2.16,73,18,5.40,GS,1698002866
4,4,leitchfield,37.4800,-86.2939,17.32,59,0,0.89,US,1698002866


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
#%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values

#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
54,54,siwa oasis,29.2041,25.5195,25.77,37,0,3.34,EG,1698002874
139,139,tsiombe,-25.3000,45.4833,23.10,72,0,2.02,MG,1698002888
183,183,at taj,24.2049,23.2857,24.08,19,0,4.24,LY,1698002896
203,203,kiziltepe,37.1933,40.5850,21.20,43,0,2.06,TR,1698002904
222,222,kupang,-10.1667,123.5833,25.32,83,0,2.06,ID,1698002909
230,230,ceres,37.5949,-120.9577,24.43,62,0,4.12,US,1698002754
234,234,al bawiti,28.3492,28.8659,23.70,48,0,4.11,EG,1698002911
284,284,long beach,33.7670,-118.1892,25.37,67,0,4.12,US,1698002873
300,300,mocimboa,-11.3167,40.3500,23.99,85,0,3.64,MZ,1698002923
342,342,mersa matruh,31.3525,27.2453,22.96,70,0,3.57,EG,1698002933


### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
54,siwa oasis,EG,29.2041,25.5195,37,
139,tsiombe,MG,-25.3000,45.4833,72,
183,at taj,LY,24.2049,23.2857,19,
203,kiziltepe,TR,37.1933,40.5850,43,
222,kupang,ID,-10.1667,123.5833,83,
230,ceres,US,37.5949,-120.9577,62,
234,al bawiti,EG,28.3492,28.8659,48,
284,long beach,US,33.7670,-118.1892,67,
300,mocimboa,MZ,-11.3167,40.3500,85,
342,mersa matruh,EG,31.3525,27.2453,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
siwa oasis - nearest hotel: فندق الكيلانى
tsiombe - nearest hotel: No hotel found
at taj - nearest hotel: No hotel found
kiziltepe - nearest hotel: Dunaysır Otel
kupang - nearest hotel: HOTEL SALUNGA
ceres - nearest hotel: Howard Johnson by Wyndham Modesto Ceres
al bawiti - nearest hotel: Old Oasis Hotel
long beach - nearest hotel: Westin
mocimboa - nearest hotel: Vumba
mersa matruh - nearest hotel: صقور
robbah - nearest hotel: No hotel found
letlhakeng - nearest hotel: No hotel found
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
rustenburg - nearest hotel: Travellers Inn
kununurra - nearest hotel: Kununurra Country Club Resort
ghardaia - nearest hotel: فندق بلعجال
worland - nearest hotel: Town House Motor Inn
morondava - nearest hotel: Hotel Sharon Menabe
mpanda - nearest hotel: Two in One Kirari Executive Lodge


,City,Country,Lat,Lng,Humidity,Hotel Name
54,siwa oasis,EG,29.2041,25.5195,37,فندق الكيلانى
139,tsiombe,MG,-25.3000,45.4833,72,No hotel found
183,at taj,LY,24.2049,23.2857,19,No hotel found
203,kiziltepe,TR,37.1933,40.5850,43,Dunaysır Otel
222,kupang,ID,-10.1667,123.5833,83,HOTEL SALUNGA
230,ceres,US,37.5949,-120.9577,62,Howard Johnson by Wyndham Modesto Ceres
234,al bawiti,EG,28.3492,28.8659,48,Old Oasis Hotel
284,long beach,US,33.7670,-118.1892,67,Westin
300,mocimboa,MZ,-11.3167,40.3500,85,Vumba
342,mersa matruh,EG,31.3525,27.2453,70,صقور


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)